In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import PyCrystalField as cef #import CEF_calculations as cef

 *******************************************************
 *                PyCrystalField 2.3.5                 *
 *  Please cite  J. Appl. Cryst. (2021). 54, 356-362   * 
 *    <https://doi.org/10.1107/S160057672001554X>      *
 *******************************************************



In [2]:
import numpy as np

experimental_data = {
    'peak_data': [
        {'T': 6, 'E': 9.26, 'Intensity': 0.986, 'Delta_E': 0.834},
        {'T': 6, 'E': 15.4, 'Intensity': 1.000, 'Delta_E': 1.23},
        {'T': 6, 'E': 18.5, 'Intensity': 0.006, 'Delta_E': 0.838},
        {'T': 6, 'E': 25.0, 'Intensity': 0.009, 'Delta_E': 1.20},
        {'T': 6, 'E': 31.4, 'Intensity': 0.010, 'Delta_E': 1.25},
        {'T': 50, 'E': 9.35, 'Intensity': 0.98, 'Delta_E': 1.05},
        {'T': 50, 'E': 15.4, 'Intensity': 1, 'Delta_E': 1.45},
        {'T': 50, 'E': 31.4, 'Intensity': 0.007, 'Delta_E': 1.35},
        {'T': 150, 'E': 10.0, 'Intensity': 1, 'Delta_E': 2.54},
        {'T': 150, 'E': 15.6, 'Intensity': 0.55, 'Delta_E': 2.11}
    ],
    'background_intensity': 0.0005,  # 0.05% of max intensity
    'magnetization_data': np.array([
        [0.7, 0.12153],
        [2, 0.35065],
        [4, 0.70129],
        [5, 0.87251],
        [7, 1.21057],
    ]),
    'susceptibility_data': np.array([
        [10.1723, 0.09164],
        [20.1009, 0.08989],
        [40.4901, 0.08192],
        [50.4605, 0.07621],
        [77.4519, 0.06115],
        [98.1425, 0.05212],
        [155.97099, 0.03669],
        [194.77699, 0.03071],
        [220.252, 0.02814],
    ]),
}

max_magnetization = np.max(experimental_data['magnetization_data'][:, 1])
max_susceptibility = np.max(experimental_data['susceptibility_data'][:, 1])

experimental_data['magnetization_data'][:, 1] /= max_magnetization
experimental_data['susceptibility_data'][:, 1] /= max_susceptibility
def Cal_M_c(cefobject,temp,field_array):
    M_array=[]
    for H in field_array:         
            t=[temp]
            cefobject=cefobject
            M_x,M_y,M_z=-1*cefobject.magnetization( 'Tm3+', t,[0,0,H])[0]
            M_array.append(M_z)
    return M_array

def get_transition_intensities(CefLevelObject, i, temperature):
    finals = range(0, 13)
    intensities = np.zeros(13)
    for idx, final_state in enumerate(finals):
        intensity = CefLevelObject.transitionIntensity(0, final_state, temperature)
        intensities[idx] = intensity
    return intensities/np.max(intensities)

def calculate_eigenvalue_penalty(calculated_eigenvalues, must_have_transitions, delta=1):
    # Calculate the energy levels and eigenvalues
    eigenvalues = calculated_eigenvalues
    
    # Calculate the differences between adjacent eigenvalues
    eigenvalue_differences = np.diff(eigenvalues) 
    # Initialize the penalty
    penalty = 0   
    # Iterate over must-have transitions
    for transition in must_have_transitions:
        min_difference = np.min(np.abs(eigenvalue_differences - transition))
        
        # Add the penalty for the current transition
        penalty += (delta - min_difference) if min_difference < delta else 0
        
    return penalty



def model_predictions(Blm_params):
    ion = 'Tm3+'
    ionJ = cef.Jion[ion][2]
    i=0
    Tm_O = []
    for n,m in [(2,0),(4,0),(4,3),(6,0),(6,3),(6,6)]:
        
        Tm_O.append(cef.StevensOp(ionJ, n, m))
        i += 1

    CefLevelObject = cef.CFLevels(Tm_O, Blm_params)
    CefLevelObject.diagonalize()
        
    eigenvalues, eigenvectors=CefLevelObject.eigenvalues,CefLevelObject.eigenvectors
     # Calculate the eigenvalues, eigenvectors, and transition intensities at different temperatures

    transition_intensities_6K, transition_intensities_50K, transition_intensities_150K = get_transition_intensities(CefLevelObject,0, 6), get_transition_intensities(CefLevelObject,0, 50), get_transition_intensities(CefLevelObject,0, 150)
    
    
    # Normalize calculated magnetization data
    # Normalize calculated magnetization data
    calculated_magnetization = Cal_M_c(CefLevelObject, 10, experimental_data['magnetization_data'][:, 0])
    normalized_calculated_magnetization = calculated_magnetization / max_magnetization


    # Normalize calculated susceptibility data
    calculated_susceptibility = -CefLevelObject.susceptibility( 'Tm3+', experimental_data['susceptibility_data'][:, 0],  1.0, 0.1)*0.5582
    normalized_calculated_susceptibility = calculated_susceptibility / max_susceptibility
    

    return transition_intensities_6K, transition_intensities_50K, transition_intensities_150K,eigenvalues,normalized_calculated_magnetization,normalized_calculated_susceptibility


In [3]:
def loss_function(Blm_params, experimental_data, must_have_transitions, eigenvalue_penalty_weight,adaptive_weights):
    # Calculate the model predictions for transition intensities at different temperatures
    transition_intensities_6K, transition_intensities_50K, transition_intensities_150K,eigenvalues,calculated_magnetization,calculated_susceptibility = model_predictions(Blm_params)
    

    # Calculate the must-have eigenvalue penalty
    eigenvalue_penalty =calculate_eigenvalue_penalty(eigenvalues, must_have_transitions)

    peak_loss = 0
    for peak in experimental_data['peak_data']:
        T, E, Intensity, Delta_E = peak['T'], peak['E'], peak['Intensity'], peak['Delta_E']
        lower_bound, upper_bound = E - Delta_E, E + Delta_E

        if T == 6:
            transition_intensities = transition_intensities_6K
        elif T == 50:
            transition_intensities = transition_intensities_50K
        elif T == 150:
            transition_intensities = transition_intensities_150K
        else:
            raise ValueError(f"Unknown temperature: {T}")
   
       

        predicted_intensity = np.sum(transition_intensities[(eigenvalues >= lower_bound) & (eigenvalues <= upper_bound)])
    
        peak_loss += (Intensity - predicted_intensity) ** 2

    # Calculate the penalty for having intensities above the background level outside the peak ranges
    background_penalty = 0
    for T, transition_intensities in [(6, transition_intensities_6K), (50, transition_intensities_50K), (150, transition_intensities_150K)]:
        for intensity, eigenvalue in zip(transition_intensities, eigenvalues):
            if not any(lower_bound <= eigenvalue <= upper_bound for peak in experimental_data['peak_data'] if peak['T'] == T):
                background_penalty += max(0, intensity - experimental_data['background_intensity']) ** 2

   

    # Compute magnetization loss
    magnetization_loss = sum((calc_m - exp_m)**2 for calc_m, exp_m in zip(calculated_magnetization, experimental_data['magnetization_data'][:,1]))

    # Compute susceptibility loss
    susceptibility_loss = sum((calc_chi - exp_chi)**2 for calc_chi, exp_chi in zip(calculated_susceptibility, experimental_data['susceptibility_data'][:,1]))

     # Combine the data and penalty terms with adaptive weights
        
#     print(peak_loss,magnetization_loss,background_penalty,susceptibility_loss,eigenvalue_penalty)

    total_loss = (
        adaptive_weights['peak_loss'] * peak_loss +
        adaptive_weights['background_penalty'] * background_penalty +
        eigenvalue_penalty_weight * eigenvalue_penalty +
        adaptive_weights['magnetization_loss'] * magnetization_loss +
        adaptive_weights['susceptibility_loss'] * susceptibility_loss
    )

    return total_loss
   


In [4]:
def gradient_of_loss_function(Blm_params, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights, delta=1e-6, p=0.1):
    grad = np.zeros_like(Blm_params)

    for i in range(len(Blm_params)):
        perturbation = p * Blm_params[i]
        if np.abs(perturbation) < delta:
            perturbation = np.sign(perturbation) * delta

        Blm_params_plus = Blm_params.copy()
        Blm_params_plus[i] += perturbation
        Blm_params_minus = Blm_params.copy()
        Blm_params_minus[i] -= perturbation
    
        loss_plus = loss_function(Blm_params_plus, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)
        loss_minus = loss_function(Blm_params_minus, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)

        grad[i] = (loss_plus - loss_minus) / (2 * perturbation)
        

    return grad

def multi_start_optimization(num_starts, lower_bound, upper_bound, num_iterations, adaptive_weights):
    best_loss = np.inf
    best_Blm_params = None

    for start in range(num_starts):
        Blm_params =np.array([-0.00344, -0.00615, -0.095, -4.14e-05, 0.000762, -0.000634])#np.random.uniform(lower_bound, upper_bound, size=6)
        m = np.zeros(6)
        v = np.zeros(6)
        print(f'Start {start + 1} of {num_starts}')

        for t in range(1, num_iterations + 1):
            grad = gradient_of_loss_function(Blm_params, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)
            m = beta1 * m + (1 - beta1) * grad
            v = beta2 * v + (1 - beta2) * (grad ** 2)
            m_hat = m / (1 - beta1 ** t)
            v_hat = v / (1 - beta2 ** t)
            # Update scaling factors based on the current Blm_params
            max_param_value = np.max(np.abs(Blm_params))
            scaling_factors = Blm_params / max_param_value
            
            Blm_params = Blm_params - learning_rate  * m_hat / (np.sqrt(v_hat) + epsilon)
 
            if t % 5 == 0:
                current_loss = loss_function(Blm_params, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)
                print(f'  Iteration {t} of {num_iterations}: Current loss: {current_loss}')
            if t % 20 == 0:
                current_loss = loss_function(Blm_params, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)
                print(Blm_params)

        current_loss = loss_function(Blm_params, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)

        if current_loss < best_loss:
            best_loss = current_loss
            best_Blm_params = Blm_params

    return best_Blm_params, best_loss

# Set the parameters for the multi-start optimization
num_starts = 10
lower_bound = -10
upper_bound = 10
num_iterations = 1000
eigenvalue_penalty_weight = 10
beta1 = 0.9
beta2 = 0.999
learning_rate = .0001
epsilon = 1e-8

must_have_transitions=[9.4,15.4,25,3,32.52]
adaptive_weights = {
    'peak_loss': 1.0,
    'background_penalty': 0.5,
    'magnetization_loss': 1.0,
    'susceptibility_loss': 1.0
}

# Run the multi-start optimization
best_Blm_params, best_loss = multi_start_optimization(num_starts, lower_bound, upper_bound, num_iterations, adaptive_weights)
print("Best Blm parameters:", best_Blm_params)
print("Best loss:", best_loss)


In [29]:
from scipy.optimize import minimize

# Define the initial guess for the Blm parameters
x0 = np.array([-0.00344, -0.00615, -0.095, -4.14e-05, 0.000762, -0.000634])

# Define the bounds for the Blm parameters
# Set the bounds for the Blm parameters
eigenvalue_penalty_weight = 10
must_have_transitions=[9.4,15.4,25,3,32.52]
adaptive_weights = {
    'peak_loss': 1.0,
    'background_penalty': 0.5,
    'magnetization_loss': 1.0,
    'susceptibility_loss': 1.0
}
lower_bound = -1
upper_bound = 1

bounds = [(lower_bound, upper_bound) for _ in range(len(x0))]

# Define the function to minimize (which is the loss function)
def objective(x):
    return loss_function(x, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)

# Compute the initial loss
initial_loss = objective(x0)
print(f"Initial loss: {initial_loss:.4f}")

# Use the minimize function to minimize the loss function
result = minimize(objective, x0, bounds=bounds)

# Extract the best set of Blm parameters from the result object
best_Blm_params = result.x

# Evaluate the loss function using the best set of Blm parameters
best_loss = objective(best_Blm_params)

# Print the final loss and best set of Blm parameters
print(f"Final loss: {best_loss:.4f}")
print(f"Best Blm parameters: {best_Blm_params}")




Initial loss: 19.0506
Final loss: 8.5244
Best Blm parameters: [-3.43858908e-03 -6.12039283e-03 -9.50313833e-02 -4.59861245e-05
  7.16453024e-04 -6.58551015e-04]


In [32]:
best_Blm_params

array([-3.43858908e-03, -6.12039283e-03, -9.50313833e-02, -4.59861245e-05,
        7.16453024e-04, -6.58551015e-04])

In [31]:
Blm_params=best_Blm_params
ion = 'Tm3+'
ionJ = cef.Jion[ion][2]
i=0
Tm_O = []
for n,m in [(2,0),(4,0),(4,3),(6,0),(6,3),(6,6)]:

    Tm_O.append(cef.StevensOp(ionJ, n, m))
    i += 1

CefLevelObject = cef.CFLevels(Tm_O, Blm_params)
CefLevelObject.diagonalize()
CefLevelObject.diagonalize()
CefLevelObject.printEigenvectors()
CefLevelObject.gsExpectation()


 Eigenvalues 	 Eigenvectors
		--------------------------------------------------------------------------------------------------
0.00000 	|  [ 0.529  0.     0.    -0.233  0.     0.     0.576  0.     0.     0.233
  0.     0.     0.529]  |
8.85447 	|  [ 0.704  0.     0.    -0.071  0.     0.     0.     0.     0.    -0.071
  0.     0.    -0.704]  |
16.62991 	|  [ 0.     0.     0.301  0.     0.    -0.847  0.     0.    -0.367  0.
  0.    -0.239  0.   ]  |
16.62991 	|  [ 0.     0.239  0.     0.    -0.367  0.     0.     0.847  0.     0.
  0.301  0.     0.   ]  |
20.61455 	|  [-0.469  0.     0.    -0.29   0.     0.     0.626  0.     0.     0.29
  0.     0.    -0.469]  |
30.71932 	|  [ 0.     0.     0.433  0.     0.    -0.267  0.     0.     0.835  0.
  0.     0.211  0.   ]  |
30.71932 	|  [ 0.     0.211  0.     0.    -0.835  0.     0.    -0.267  0.     0.
 -0.433  0.     0.   ]  |
38.69645 	|  [-0.071  0.     0.    -0.704  0.     0.     0.     0.     0.    -0.704
  0.     0.     0.071]  |
103.3

In [ ]:
import numpy as np
from scipy.optimize import basinhopping

# Define the objective function to minimize
def objective(params):
    return loss_function(params, experimental_data, must_have_transitions, eigenvalue_penalty_weight, adaptive_weights)

# Define the bounds for the Blm parameters
eigenvalue_penalty_weight = 4
must_have_transitions=[9.4,15.4,25,3,32.52]
adaptive_weights = {
    'peak_loss': 1.0,
    'background_penalty': 0.5,
    'magnetization_loss': 1.0,
    'susceptibility_loss': 1.0
}
lower_bound = -.5
upper_bound = .5
bounds = [(lower_bound, upper_bound) for _ in range(6)]

# Define the initial guess for the Blm parameters
x0 = np.array([-0.00344, -0.00615, -0.095, -4.14e-05, 0.000762, -0.000634])

# Define the cooling schedule for the simulated annealing algorithm
def schedule(t):
    return 1.0 / np.log(1.0 + t)

# Run the simulated annealing optimization
result = basinhopping(objective, x0, minimizer_kwargs={'bounds': bounds}, niter=1000, T=1.0, stepsize=0.5, callback=lambda x, f, accept: print(f"Current loss: {f:.4f}"))

# Print the best solution found
print("Best Blm parameters:", result.x)
print("Best loss:", result.fun)


Current loss: 6.6586
Current loss: 6.0006
Current loss: 7.2518
Current loss: 5.1753
Current loss: 5.1716
Current loss: 69.8937
Current loss: 8.5134
Current loss: 6.6502
Current loss: 5.8508
Current loss: 10.1316
Current loss: 5.6350
Current loss: 8.2626
Current loss: 14.9836
Current loss: 10.7626
Current loss: 6.1608
Current loss: 7.1718
Current loss: 6.1128
Current loss: 14.9822
Current loss: 8.5462
Current loss: 6.3882
Current loss: 7.5680
Current loss: 9.3983
Current loss: 14.9821
Current loss: 14.9819
Current loss: 14.9819
Current loss: 7.6331
Current loss: 14.9775
Current loss: 5.6489
Current loss: 6.0005
Current loss: 8.5757
Current loss: 7.5693
Current loss: 9.5219
Current loss: 6.0979
Current loss: 14.9807
Current loss: 69.8971
Current loss: 6.2129
Current loss: 8.1458
Current loss: 6.5395
Current loss: 9.6816
Current loss: 8.6543
Current loss: 6.3025
Current loss: 7.5436
Current loss: 5.1987
Current loss: 6.2019
Current loss: 5.7271
Current loss: 69.7508
Current loss: 9.9674
C